# Minutenverlauf

## Bibliotheken einbinden
(und magische Kommandos für jupyter)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

%config Completer.use_jedi = False
%matplotlib widget

In [ ]:
# Möglicherweise weitere Schriftarten laden (hier: Standardschriftart ändern)
from matplotlib import rcParams
#rcParams['axes.spines.top'] = False
#rcParams['axes.spines.right'] = False

import matplotlib.font_manager
import os

# Add every font at the specified location
font_dir = [os.path.join(os.getcwd(),"..","fonts")]
for font in matplotlib.font_manager.findSystemFonts(font_dir):
    print(font)
    matplotlib.font_manager.fontManager.addfont(font)
# Debugging: Prüfen auf eine Schrift
#print(matplotlib.font_manager.findfont('Caveat'))
# Debugging: Fonts listen
#matplotlib.font_manager.findSystemFonts(fontpaths=font_dir, fontext='ttf')[:10]

# Set font family globally
rcParams['font.family'] = 'Caveat'
rcParams.update({'font.size': 12})


## Daten einlesen

In [ ]:
# Laden der minütlichen Information
#fname = 'example_data/example.csv'
fname = '930450-Hauptverteiler-1MIN.csv'
data = pd.read_csv(fname, delimiter=",",index_col=0, parse_dates=[0], na_values=['No CT'])
#data

## Daten eingrenzen

Wir beschränken die verfügbaren Daten zunächst auf einen Zeitraum:

In [ ]:
plotdata = data["2022-07-09":"2022-07-13"]

Da wir die Daten später gruppieren möchten, wandeln wir die Index-Spalte (momentan die Zeit) in eine normale Spalte um.

In [ ]:
plotdata = plotdata.reset_index()

## Daten gruppieren
Gruppiert wird nun auf Tag und Stunde.

In [ ]:
plotdata.iloc[:,0:]

In [ ]:
# Nur die drei Phasen des Hauptverteilers aus den Daten selektieren
bdata = plotdata.iloc[:,0:]
# Alle Verbrauchsstellen summieren
bdata = bdata.assign(Verbraucher=lambda x: x.iloc[:,4:].sum(1))
# Neue Spalte der Summe der drei Phasen, da wir Gesamtverbrauch wissen wollen.
bdata = bdata.assign(HV=lambda x: x.iloc[:,1:4].sum(1))
# Alles was nicht von Verbrauchsstellen gemessen wurde
bdata = bdata.assign(Rest=lambda x: x['HV']-x['Verbraucher'])

# Garage
bdata = bdata.assign(Garage=lambda x: x.iloc[:,4:7].sum(1))
# UV
bdata = bdata.assign(UV=lambda x: x.iloc[:,7:10].sum(1))
# Mikrowelle
bdata.rename(columns={ bdata.columns[10]: "Mikrowelle" }, inplace = True)
# Arbeitszimmer
bdata.rename(columns={ bdata.columns[11]: "Arbeitszimmer" }, inplace = True)
# Sina
bdata.rename(columns={ bdata.columns[12]: "Sina" }, inplace = True)
# Malte
bdata.rename(columns={ bdata.columns[13]: "Malte" }, inplace = True)
# Studio
bdata.rename(columns={ bdata.columns[14]: "Studio" }, inplace = True)
# Lüftung
bdata.rename(columns={ bdata.columns[15]: "Lueftung" }, inplace = True)
# Geschirrspühler
bdata.rename(columns={ bdata.columns[16]: "Geschirr" }, inplace = True)
# Heizung
bdata.rename(columns={ bdata.columns[17]: "Heizung" }, inplace = True)
# Backofen
bdata.rename(columns={ bdata.columns[18]: "Backofen" }, inplace = True)
# Kühlschrank
bdata.rename(columns={ bdata.columns[19]: "Kuehlschrank" }, inplace = True)

# Neue Spalte für spätere Gruppierung: Tag des Monats
#bdata = bdata.assign(Day=lambda x: x.iloc[:,0].dt.day)
# Neue Spalte für spätere Gruppierung: Stunde des Tages
#bdata = bdata.assign(Hour=lambda x: x.iloc[:,0].dt.hour)

# Nach Tag und Stunde gruppieren
#bdata = bdata.groupby(["Day","Hour"])[['Garage','UV','Mikrowelle','Arbeitszimmer','Sina','Malte','Studio','Lueftung','Geschirr','Heizung','Backofen','Kuehlschrank','Rest']].mean()
# MultiIndex auflösen, indem neue Namen für die Indexspalten vergeben werden
#bdata.index = bdata.index.set_names(['Day', 'Hour'])
# Indexspalten in normale Spalten konvertieren
#bdata = bdata.reset_index()

# Nach Tag gruppieren
#bdata = bdata.groupby(["Day"]).sum()
#bdata.drop(columns=['Hour'],inplace=True)

#plt.plot(test.index.day - 1 + (test.index.hour)/24.0,test.Temperature.rolling(7).mean(),'-', label="Temperaturverlauf Monat %i" % (i))
bdata

### Daten als gestapelte Balken

In [ ]:
h = plt.figure(figsize=(15, 10))
ax = h.gca()
ax.grid()
plt.plot(bdata.iloc[:,0],bdata.HV.rolling(15).mean(),'k-', label="HV")
plt.plot(bdata.iloc[:,0],bdata.UV.rolling(15).mean(),'b-', label="UV")
plt.plot(bdata.iloc[:,0],bdata.Garage.rolling(15).mean(),'r-', label="Garage")
plt.plot(bdata.iloc[:,0],bdata.Geschirr.rolling(15).mean(),'y-', label="Geschirr")
ax.legend()

#bdata.plot.bar(stacked=True, rot=90, ax=ax)
# Hilfsvariable für Anzeige
#zeitraum = "Vom %s bis %s" % (plotdata.iloc[:,0].min().strftime('%d.%m.%Y'),plotdata.iloc[:,0].max().strftime('%d.%m.%Y'))
# Ploteinstellungen
#ax.set_title("Verbrauch pro Tag aller Messstellen\n" + zeitraum, size=15)
#ax.set_xlabel("Tag")
#ax.set_ylabel("Stromverbrauch in kW")
# Speichern
#fname = "%s-%s-Messstellen_pro_Tag.pdf" % (plotdata.iloc[:,0].min().strftime('%Y%m%d'),plotdata.iloc[:,0].max().strftime('%Y%m%d'))
#h.savefig(fname)